<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
import sklearn as sk

1. Carica in un dataframe il file “bank.csv” (preso da kaggle -
https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers) che contiene dati relativi ai
clienti di una banca. Lo scopo è predire se il cliente abbandonerà la banca (feature Exited).
Quante righe e quante colonne sono presenti? Ci sono valori nulli? Il dataset è bilanciato rispetto
alla classe da predire?

In [16]:
df = pd.read_csv("churn.csv.xls")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.shape[0] #num righe

10000

In [5]:
df.shape[1] #num colonne

14

In [8]:
df.isnull().sum().sum()

np.int64(0)

In [7]:
#per verificare se è bilanciato o meno faccio
df['Exited'].value_counts()
#come si può notare ci sono quasi 4x il numero di 0, non è quindi bilanciato

,count
Exited,
0,7963
1,2037


2. Converti i valori di Gender in numerici (Male = 0, Female = 1).

In [17]:
df['Gender'] = df['Gender'].map({'Male' : 0, 'Female' : 1})

In [18]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,1,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,1,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,1,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0


3. Trasforma la feature Geography con OneHotEncoder.

In [22]:
from sklearn import preprocessing

In [24]:
#c'è da fare del reshape per creare input che vada bene a onehotencoder. problemi che vengono risolti nel momento in cui metteremo il onehotencoder in pipeline
enc = preprocessing.OneHotEncoder()
enc.fit(np.array(df['Geography']).reshape(-1,1))
#chiamo funzione fit su un array numpy creato dalla colonna Geography, array numpy trasformato per fare in modo che sia a due dimensionei siccome il one hot encoder
#lo richiede di 2 dimensioni. reshape necessita la forma giusta

OneHotEncoder()

In [26]:
enc.transform(np.array(df['Geography']).reshape(-1,1)) #restituisce una sparse matrix voglio ottenere dataframe da attaccare a quello inziale

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [27]:
#devo ottenere un dataframe in modo che queste nuove colonne create dal one hot encoder  vengano attaccate al dataframe di partenza da cui ho tolto geography
pd.DataFrame.sparse.from_spmatrix(enc.transform(np.array(df['Geography']).reshape(-1,1)))

,0,1,2
0,1.0,0,0
1,0,0,1.0
2,1.0,0,0
3,1.0,0,0
4,0,0,1.0
...,...,...,...
9995,1.0,0,0
9996,1.0,0,0
9997,1.0,0,0
9998,0,1.0,0


In [28]:
#più semplicemente si può fare con getdummies
df = pd.get_dummies(df,columns=['Geography'],dtype=int)

In [29]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,0,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,0,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,1,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,0,42,3,75075.31,2,1,0,92888.52,1,0,1,0


4. Elimina le feature RowNumber, CustomerId, Surname e dividi il dataset in train (75% del
dataset) e test (25%). Usa random_state=0.

In [31]:
from sklearn.model_selection import train_test_split

In [30]:
#si tratta di valori di colonne univoci che servono ad identificare il clitente, non aiutano quindi la predizione dell'algoritmo.
#nel train test-split oltre ad eliminare il target, occorre anche eliminare queste 3 colonne
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True) #con inplace=true il dataframe viene trasformato direttamente

In [32]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('Exited',axis=1),df['Exited'],test_size=0.25,random_state=0,stratify=df['Exited'])
#necessario droppare la lable dalle X, df['exited'] è la lable
#stratify = colonna target fa si che il dataset sia diviso con la stessa prop di train e test che c'è nel dataset iniziale

5. Allena un decision tree per predire l’abbandono dei clienti, scegli tu i valori dei parametri e
calcola l’accuratezza della predizione.

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [35]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test,y_pred)

0.7956

6. Aggiungi una nuova feature nel dataset con il valore di (EstimatedSalary*Tenure+Balance)/2.
L’accuratezza della predizione migliora?

In [36]:
df['New_Feature'] = (df['EstimatedSalary'] * df['Tenure'] + df['Balance']) / 2
X_train,X_test,y_train,y_test = train_test_split(df.drop('Exited',axis=1),df['Exited'],test_size=0.25,random_state=0,stratify=df['Exited'])
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test,y_pred)
#accuracy migliora molto poco

0.7984

7. Applica due trasformazioni diverse a tutto il dataset e/o a feature specifiche, valuta la predizione.


In [37]:
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance',
                    'NumOfProducts', 'EstimatedSalary', 'New_Feature']
X = df.drop('Exited', axis=1)
y = df['Exited']
scaler_std = preprocessing.StandardScaler()
X[numeric_features] = scaler_std.fit_transform(X[numeric_features])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test,y_pred)
#aumenta l'accuracy di un qualcosina, si può applicare altra trasf e vedere come cambia

0.8008